In [1]:
import numpy as np
from keras.datasets import imdb
from tensorflow.python.keras.models import Model, load_model, Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Embedding, Input, Dot, Flatten, Layer, Lambda
from keras_preprocessing.sequence import pad_sequences
from tensorflow.python.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import pickle
import re
import os
from keras.utils import to_categorical
from datetime import datetime

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
placement_weights = np.array([1, 0.875, 0.75, 0.625, 0.375, 0.25, 0.125, 0])

In [4]:
with open('embeddings_100.pkl', 'rb') as file:
    weights = pickle.load(file)
word2idx = {}
with open('word2idx.pkl', 'rb') as file:
    word2idx = pickle.load(file)

In [5]:
sentences = []
with open('teams.txt', 'r') as file:
    for line in file:
        sentences.append(line.strip().split())

In [6]:
maxWords = 0
for i in range(len(sentences)):
    if maxWords < len(sentences[i]):
        maxWords = len(sentences[i])

In [7]:
X = np.zeros((len(sentences), maxWords - 1, 100))
Y = np.zeros((len(sentences)))
for i in range(len(sentences)):
    for j in range(len(sentences[i])):
        if sentences[i][j] == 'nan':
            X[i][j] = np.zeros((100,))
        else:
            if str(sentences[i][j]).isdigit():
                Y[i] = placement_weights[int(sentences[i][j]) - 1]
            else:
                X[i][j] = weights[word2idx[sentences[i][j]] - 1]

In [8]:
X_train, X_rest, Y_train, Y_rest = train_test_split(X, Y, test_size=0.1, random_state=42)
X_test, X_val, Y_test, Y_val = train_test_split(X_rest, Y_rest, test_size=0.5, random_state=42)

In [9]:
rnn_model = Sequential()
rnn_model.add(LSTM(200, dropout=0.5, recurrent_dropout=0.5))
rnn_model.add(Dense(1024, activation='relu'))
rnn_model.add(Dense(512, activation='relu'))
rnn_model.add(Dense(256, activation='relu'))
rnn_model.add(Dense(128, activation='relu'))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(16, activation='relu'))
rnn_model.add(Dense(1))

rnn_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'accuracy'])

checkpoint = []
date = ('_' + str(datetime.now().day) + '-' + str(datetime.now().month) + '-' + str(datetime.now().year) + '_' +
        str(datetime.now().hour) + '-' + str(datetime.now().minute).zfill(2))
checkpoint.append(ModelCheckpoint(filepath='rnn_tft_val_mse' + date + '.h5',
                             monitor='val_mse',
                             verbose=1,
                             save_best_only=True
                             ))
checkpoint.append(ModelCheckpoint(filepath='rnn_tft_val_loss' + date + '.h5',
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True
                             ))

In [10]:
rnn_model.fit(X_train, Y_train, validation_data=(X_val, Y_val), epochs=1000, batch_size=4096, callbacks=checkpoint)

Epoch 1/1000
15/15 [==============================] - 9s 401ms/step - loss: 0.2745 - mse: 0.2745 - accuracy: 0.1305 - val_loss: 0.2273 - val_mse: 0.2273 - val_accuracy: 0.1191

Epoch 00001: val_mse improved from inf to 0.22733, saving model to rnn_tft_val_mse_27-4-2024_15-20.h5

Epoch 00001: val_loss improved from inf to 0.22733, saving model to rnn_tft_val_loss_27-4-2024_15-20.h5
Epoch 2/1000
15/15 [==============================] - 6s 375ms/step - loss: 0.1569 - mse: 0.1569 - accuracy: 0.1290 - val_loss: 0.1209 - val_mse: 0.1209 - val_accuracy: 0.1280

Epoch 00002: val_mse improved from 0.22733 to 0.12089, saving model to rnn_tft_val_mse_27-4-2024_15-20.h5

Epoch 00002: val_loss improved from 0.22733 to 0.12089, saving model to rnn_tft_val_loss_27-4-2024_15-20.h5
Epoch 3/1000
15/15 [==============================] - 6s 369ms/step - loss: 0.1141 - mse: 0.1141 - accuracy: 0.1453 - val_loss: 0.1095 - val_mse: 0.1095 - val_accuracy: 0.1872

Epoch 00003: val_mse improved from 0.12089 to 0

In [11]:
rnn_model.evaluate(X_test, Y_test, verbose=1)

106/106 [==============================] - 3s 31ms/step - loss: 0.0706 - mse: 0.0706 - accuracy: 0.2342


[0.07061946392059326, 0.07061946392059326, 0.23417533934116364]

In [21]:
team8 = 'TFT11_FortuneYord nan nan nan TFT11_Neeko TFT11_Item_FortuneEmblemItem nan nan TFT11_Illaoi nan nan nan TFT11_Zoe nan nan nan TFT11_Tristana nan nan nan TFT11_Annie TFT_Item_Redemption TFT_Item_RedBuff TFT_Item_GargoyleStoneplate TFT11_Galio TFT_Item_GiantsBelt nan nan TFT11_Syndra nan nan nan TFT11_Azir TFT_Item_HextechGunblade TFT_Item_ArchangelsStaff TFT_Item_JeweledGauntlet TFT6_Augment_Distancing TFT11_Augment_FortuneCrest TFT10_Augment_GoingLong'

In [22]:
team8 = team8.split()
vector = np.zeros((1,len(team8), 100))
for i in range(len(team8)):
    if team8[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    else:
        vector[0][i] = weights[word2idx[team8[i]] - 1]

In [23]:
rnn_model.predict(vector)

array([[0.8434153]], dtype=float32)

In [24]:
team1 = 'TFT11_Caitlyn nan nan nan TFT11_Lux TFT_Item_RapidFireCannon TFT_Item_HextechGunblade nan TFT11_Shen nan nan nan TFT11_Ashe TFT_Item_GuinsoosRageblade TFT_Item_HextechGunblade TFT_Item_MadredsBloodrazor TFT11_Ornn TFT_Item_ThiefsGloves TFT_Item_PowerGauntlet TFT_Item_LastWhisper TFT11_Nautilus nan nan nan TFT11_Sett TFT_Item_AdaptiveHelm TFT_Item_Redemption nan TFT11_Lissandra TFT_Item_Leviathan TFT_Item_JeweledGauntlet TFT_Item_RecurveBow TFT11_Hwei nan nan nan TFT11_Augment_WanderingTrainerGold TFT9_Augment_Commander_Ascension TFT10_Augment_InspiringEpitaph'


In [25]:
team1 = team1.split()
vector = np.zeros((1,len(team1), 100))
for i in range(len(team1)):
    if team1[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    else:
        vector[0][i] = weights[word2idx[team1[i]] - 1]

In [26]:
rnn_model.predict(vector)

array([[1.0332756]], dtype=float32)

In [18]:
team5 = 'TFT11_Caitlyn nan nan nan TFT11_Aatrox nan nan nan TFT11_Shen nan nan nan TFT11_Illaoi TFT_Item_Redemption TFT_Item_GargoyleStoneplate TFT_Item_WarmogsArmor TFT11_Morgana TFT_Item_SpearOfShojin TFT5_Item_ArchangelsStaffRadiant TFT_Item_HextechGunblade TFT11_Kayn nan nan nan TFT11_Sylas nan nan nan TFT11_Sett TFT_Item_GuinsoosRageblade TFT_Item_GuardianAngel nan TFT11_Irelia TFT_Item_RapidFireCannon TFT_Item_LastWhisper TFT_Item_InfinityEdge TFT7_Augment_BirthdayPresents TFT9_Augment_Harmacist2 TFT9_Augment_YoungAndWildAndFree'

In [19]:
team5 = team5.split()
vector = np.zeros((1,len(team5), 100))
for i in range(len(team5)):
    if team5[i] == 'nan':
        vector[0][i] = np.zeros((100,))
    else:
        vector[0][i] = weights[word2idx[team5[i]] - 1]

In [20]:
rnn_model.predict(vector)


array([[0.971196]], dtype=float32)